In this notebook we will deploy the logistic regression model (that we have trained in notebook
**"02_customer_churn_train_logistic_regression.ipynb"**) as a real time service that can be called via Azure ML SDK or Rest request to make a prediction on a new data point.

To this end, we will deploy the model as a web service hosted in a container.

In general, in Azure ML you can deploy a model either with Azure Container Instances (ACI) or Azure Kubernetes Service (AKS). 

ACI is only suited for development and testing purposes.

We will deploy our model on an Azure Kubernetes Service (AKS) cluster and will enable required authentication. This will require REST requests to include an authorization header.

We'll deploy the model as a service named **churn-prediction-service-1**.
The deployment process includes the following steps:

1. Define an inference configuration, which includes the scoring and environment files required to load and use the model.
2. Define a deployment configuration that defines the execution environment in which the service will be hosted.
   In this case, an Azure Kubernetes Cluster.
3. Deploy the model as a web service.
4. Verify the status of the deployed service.

Deployment will take some time as it first runs a process to create a container image, and then runs a process 
to create a web service based on the image. When deployment has completed successfully, you'll see a status of Healthy.

# Notebook Setup

In [1]:
## Import libraries
# Standard libraries
import os
import shutil

# Azure ML SDK libraries
from azureml.core import Model, Workspace, Environment
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.model import InferenceConfig
from azureml.core.image import ContainerImage
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

# Azure ML Setup

In [2]:
# Load the workspace from the config file. If this notebook runs in Azure ML, from_config() will do the job.
ws = Workspace.from_config()
print("Ready to use Azure ML SDK {} to work with {}".format(azureml.core.VERSION, ws.name))

Ready to use Azure ML SDK 1.0.85 to work with wschurnazuremldemo


# List Models from Workspace

In [3]:
for model in Model.list(ws):
    print(model.name, "version:", model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ("\t",tag_name, ":", tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ("\t",prop_name, ":", prop)
    print("\n")

customer_churn_dt_model version: 1
	 Training Context : Notebook
	 Algorithm : Decision Tree


customer_churn_log_reg_model version: 1
	 Training Context : Estimator
	 Algorithm : Logistic Regression
	 AUC : 0.7065752294266209
	 Accuracy : 0.7998580553584103
	 Precision : 0.8240495137046862
	 Recall : 0.9182266009852217
	 F Score : 0.8685927306616963




# Retrieve Model for Deployment

In [4]:
# We will deploy our logistic regression model here
model = ws.models["customer_churn_log_reg_model"] # by default latest version
print(model.name, "version", model.version)

customer_churn_log_reg_model version 1


# Prepare Folder and Score File to Host Web Service for Deployment

In [5]:
service_folder = "model_service"

# Create a folder for the web service files
experiment_folder = './' + service_folder
os.makedirs(service_folder, exist_ok=True)

print(service_folder, "folder created.")

model_service folder created.


In [6]:
# Create folder "utilities"
os.makedirs("utilities", exist_ok=True)

# Copy the utility files into the experiment folder
os.makedirs(service_folder + "/utilities", exist_ok=True)
shutil.copy("utilities/preprocess_dataset.py", os.path.join(service_folder, "utilities/preprocess_dataset.py"))

'model_service/utilities/preprocess_dataset.py'

In [7]:
%%writefile $service_folder/score.py

# Import preprocessing function which is defined in utilities folder
import utilities.preprocess_dataset

# Install libraries
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model
from azureml.core import Workspace
from sklearn.linear_model import LogisticRegression
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'customer_churn_log_reg_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)["data"])
    # Preprocess the data
    data_processed = utilities.preprocess_dataset.run(data, training_mode=False)
    # Get the prediction and the corresponding probabilities from the model
    predictions = model.predict(data_processed)
    prediction_probas = model.predict_proba(data_processed)
    # Get the corresponding classnames for each prediction (0 or 1)
    classnames = ["Non Churn", "Churn"]
    predicted_classes = []
    predicted_classes_probas = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    for prediction_proba in prediction_probas:
        predicted_classes_probas.append(list(prediction_proba))
    # Return the predictions and their corresponding probabilities as JSON
    return json.dumps({"predicted_classes":predicted_classes, "predicted_classes_probas":predicted_classes_probas})

Writing model_service/score.py


# Create Dependencies and Inference Config

In [8]:
# Create dependencies and build a new environment with these dependencies
conda_deps = CondaDependencies.create(conda_packages=['numpy','scikit-learn==0.22.1'],
                                      pip_packages=['azureml-defaults','azureml-core'])
myenv = Environment(name='myenv')
myenv.python.conda_dependencies = conda_deps

# use an image available in public Container Registry without authentication
# myenv.docker.base_image = "mcr.microsoft.com/azureml/o16n-sample-user-base/ubuntu-'miniconda"
# myenv.inferencing_stack_version='latest'

In [9]:
# Create inference config
inf_config = InferenceConfig(source_directory=service_folder, 
                             entry_script='score.py',
                             environment=myenv)

# Provision Azure Kubernetes Service

In [ ]:
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'aks-1' 

# Create the cluster if not already created
try:
    aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)
except:
    aks_target = ComputeTarget(workspace = ws, name = aks_name)

# Wait for aks cluster
aks_target.wait_for_completion()

# Configure AKS Deployment

In [ ]:
# Option 1:
# Set the web service configuration (using default here)
# aks_config = AksWebservice.deploy_configuration()

# Option 2:
# Enable token authentication and disable (key) authentication on the webservice
aks_config = AksWebservice.deploy_configuration(token_auth_enabled=True, auth_enabled=False)

# Deploy Model

In [ ]:
aks_service_name = "churn-prediction-service-1"

aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[model],
                           inference_config=inf_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target)

aks_service.wait_for_deployment(show_output = True)

# Get service state
print(aks_service.state)

In [ ]:
# Get service logs
# print(aks_service.get_logs())

In [ ]:
# If you need to make a change and redeploy, you may need to delete the unhealthy service using the following code:
# aks_service.delete()

# List Web Services from Workspace

In [ ]:
for webservice_name in ws.webservices:
    print(webservice_name)

# Appendix

In [ ]:
'''# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn==0.22.1")

# Save the environment config as a .yml file
env_file = folder_name + "/customer_churn_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())'''